   # Simulation of the process
   We start by constructing the code that will be used in order to simulate the Itô process
   $$\text{d}X_{t}=-\nabla H\left(X\right)+\sigma\text{d}B_{t},$$
   for
   $$H(x_{1},\dots,x_{N})=-J\sum\cos(x_{i}-x_{i-1}).$$
   
   We will define it with three types of boundary – periodic, free, and the zero boundary condition.

In [ ]:
using Statistics

@enum BoundaryCondition PeriodicBoundary FreeBoundary ZeroBoundary

#=
This function receives the position vector X and the coupling constant J,
and puts in the vector force the drift term of the system.
=#
function F(X::Vector{Float64},J::Float64, force::Vector{Float64}, boundary::BoundaryCondition)
    N=size(X)[1]
    
    for i in 2:N-1
        force[i] = J*(sin(X[i-1]-X[i]) + sin(X[i+1]-X[i]))
    end
    if boundary == PeriodicBoundary
        force[1] = J*(sin(X[N]-X[1]) + sin(X[2]-X[1]))
        force[N] = J*(sin(X[N-1]-X[N]) + sin(X[1]-X[N]))
    elseif boundary == FreeBoundary
        force[1] = J* sin(X[2]-X[1])
        force[N] = J*sin(X[N-1]-X[N])
    elseif boundary ==ZeroBoundary
        force[1] = J*(sin(X[2]-X[1])- sin(X[1]))
        force[N] = J*(sin(X[N-1]-X[N])-sin(X[N]))
    end
    nothing
end
        
#=
Move the vector X a single time step dt, with the parameters J and sigma. "force" is used to store the force without allocating memory.
=#
function one_step(X::Vector{Float64},J::Float64,σ::Float64,dt::Float64,force::Vector{Float64}, boundary::BoundaryCondition)
    N=size(X)[1]
    noise = (rand(N) .- 0.5) .* (√12*σ*√dt)
    F(X,J,force,boundary)
    X .+= force .* dt .+ noise
    nothing
end
    
#=
Run the Itô process starting from X0 for time T with time discretization dt and parameters σ,J
=#
function runprocess(X0::Vector{Float64},T::Float64,dt::Float64,σ::Float64,J::Float64, boundary::BoundaryCondition = PeriodicBoundary)
    N=size(X0)[1]
    X=X0
    steps = Int(ceil(T/dt))
    force = Array(X0)
    process = zeros(steps,N)
    for s in 1:steps
        one_step(X,J,σ,dt,force,boundary)
        process[s,:]=Array(X)
    end
    process
end


Next, we define functions that will be used to compute observables we are interested in.

In [ ]:
#=
Calculate the winding number of a vector X.
=#
function windingnumber(X::Vector{Float64},boundary::BoundaryCondition = PeriodicBoundary)
    N=size(X)[1]
    phase = 0
    for i in 2:N
        phase += rem2pi(X[i]-X[i-1],RoundNearest)
    end
    if boundary == PeriodicBoundary
        phase += rem2pi(X[1]-X[N],RoundNearest)
    elseif boundary == ZeroBoundary
        phase += rem2pi(X[1],RoundNearest)
        phase += rem2pi(-X[N],RoundNearest)
    end
    phase/2π
end
    
#=
Calculate the total magnetization, given as a complex number.
=#
function total_magnetization(X::Vector{Float64})
    complex_phase = exp.(X.*1im)
    sum(complex_phase)
end    

# Plotting
We define functions that plot the configuration.

In [ ]:
using Plots
using Printf
#=
Plots a state of the system X
=#
function plotconfiguration(X, title="XY configuration")
    N = size(X)[1]
    pos=[]
    vec=[]
    for i in 1:N
        pos = [pos...,(5*cos((i-1)/N * 2π),5*sin((i-1)/N * 2π))]
        vec = [vec...,(cos(X[i]),sin(X[i]))]
    end
    quiver(pos, quiver=vec, xlim=(-6,6), ylim=(-6,6), yaxis=false, xaxis=false, size=(500,500),title = title)
end

#=
Create an animation of the process together with the winding number
=#
function animateprocess(process,frames=1000)
    steps = size(process)[1]
    winding = [windingnumber(process[i,:]) for i in 1:steps]
    jmp = round(Int,steps/frames)
    @animate for i=1:jmp:steps
        frametitle = @sprintf "t = %.3f" i*dt
        arrows = plotconfiguration(process[i,:],frametitle )
        windingnum = plot(winding[1:i],xlim=(0,steps), ylim=(-2,2), legend=false)
        plot(arrows,windingnum, layout=grid(2,1,heights=[0.8,0.2]),size=(500,500))
    end
end

# Example
Here is an example showing how to use the functions defined above.

In [ ]:
N = 20
dt = 0.0001
T = 20.0
J = 20.0
σ = 3.0

X0=[range(0,length = N, step = 2π/N)...]
#plotconfiguration(X0, "Initial configuration")

process = runprocess(X0,T,dt,σ,J,PeriodicBoundary)
steps = size(process)[1]

winding = [windingnumber(process[i,:]) for i in 1:steps]
plot((1:steps)*dt, winding, ylim = (-2,2),legend=false)

In [ ]:
anim=animateprocess(process,100)
gif(anim,fps=10)

# Exit time from the metastable state
We will simulate the time it take to change winding number. We will compare its distribution to an exponential law, to see if there is indeed a loss of memory.

In [ ]:
function simulate_exit_time(N,dt,Tmax,J,σ,winding_number::Int,boundary::BoundaryCondition=PeriodicBoundary)
    X = [range(0,length = N, step = (winding_number+0.01)*2π/N)...]  #step size 0 is not allowed, so we add a small bias to the initial configuration
    force = Array(X)
    
    stepsmax = round(Int,Tmax/dt)
    process = zeros(stepsmax,N)
    
    for s in 1:stepsmax
        one_step(X,J,σ,dt,force,boundary)
        if abs(round(Int,windingnumber(X))-winding_number) > 0.5
            return s*dt
        end
    end
    return Tmax+1
end 

In [ ]:
N = 20
dt = 0.0001
Tmax = 50.0
J = 20.0
σ = 3.0


exit_times = [simulate_exit_time(N,dt,Tmax,J,σ,1) for i in 1:1000]
println("rejection rate: ", round(mean(exit_times.>Tmax),digits=3))

λ = 1/mean(exit_times)

histogram(exit_times,bins=15,normed=true,legend=false)
t = Array(0:0.1:maximum(exit_times))
plot!(t,λ*exp.(-λ*t))

# Correlations in the metastable phase
When the winding number is $0$, we expect all rotors to align, so in particular the first rotor and the $N/2$-th rotor should be positively correlated. But in the metastable phase with winding number $1$, in the mimimum energy configuration the first and the $N/2$-th rotors point in opposite directions, so we expect negative correlation.

We can use the following code in order to verify this behavior, and see that it is indeed topological, i.e., it appears when the boundary is periodic, but with free boundary conditions the correlation is always positive.

In [ ]:
N = 20
dt = 0.0001
T = 10.0
J = 40.0
σ = 3.0

function simulate_correlation(N,dt,T,J,σ,winding_number::Int,boundary::BoundaryCondition)
    X0 = [range(0,length = N, step = (winding_number+0.01)*2π/N)...]  #step size 0 is not allowed, so we add a small bias to the initial configuration
    process = runprocess(X0,T,dt,σ,J,boundary)
    steps = size(process)[1]
    
    if boundary == PeriodicBoundary
        winding = [round(Int,windingnumber(process[i,:])) for i in 1:steps]
        if !all(winding .== winding_number)
            #println("System left the metastable state!")
            return 2
        end
    end
    
    correlation = [cos(process[i,1]-process[i,round(Int,N/2)]) for i in 1:steps]
    return mean(correlation[round(Int,steps/2):steps])
end

In [ ]:
corr = [simulate_correlation(N,dt,T,J,σ,1,PeriodicBoundary) for i in range(1,100)]
corr = filter(x -> x < 1, corr)
println("The obsereved mean correlation is ", round(mean(corr),digits=3))
#histogram(corr)

In [ ]:
corr = [simulate_correlation(N,dt,T,J,σ,0,PeriodicBoundary) for i in range(1,100)]
corr = filter(x -> x < 1, corr)
println("The obsereved mean correlation is ", round(mean(corr),digits=3))
#histogram(corr)

In [ ]:
corr = [simulate_correlation(N,dt,T,J,σ,1,FreeBoundary) for i in range(1,100)]
corr = filter(x -> x < 1, corr)
println("The obsereved mean correlation is ", round(mean(corr),digits=3))
#histogram(corr)

In [ ]:
corr = [simulate_correlation(N,dt,T,J,σ,0,FreeBoundary) for i in range(1,100)]
corr = filter(x -> x < 1, corr)
println("The obsereved mean correlation is ", round(mean(corr),digits=3))
#histogram(corr)

# Response to external field
We can also ask how the system responds to an external field $B$, i.e., adding to the Hamiltonian the term $-B\sum e^{i x_i}.$

When the winding number is $0$ the rotors are all aligned, and when adding a field $B$ they would all be aligned in the direction of the field. The phase with winding number $1$ is different – if we try to align one rotator with $B$, the rotator in the opposite location would tend to align in the opposite direction. This is the reason we expect a much smaller reaction to external field in this metastable phase.

We will check what is the mean magnetization of the system for fixed $B$ in the different phases.


In [ ]:
#Redefine all functions with external field
#=
This function receives the position vector X, the coupling constant J, and the external field B,
and puts in the vector force the drift term of the system.
=#
function F(X::Vector{Float64},J::Float64, force::Vector{Float64}, B::Float64, boundary::BoundaryCondition)
    N=size(X)[1]
    
    for i in 2:N-1
        force[i] = J*(sin(X[i-1]-X[i]) + sin(X[i+1]-X[i]))
    end
    if boundary == PeriodicBoundary
        force[1] = J*(sin(X[N]-X[1]) + sin(X[2]-X[1]))
        force[N] = J*(sin(X[N-1]-X[N]) + sin(X[1]-X[N]))
    elseif boundary == FreeBoundary
        force[1] = J* sin(X[2]-X[1])
        force[N] = J*sin(X[N-1]-X[N])
    elseif boundary ==ZeroBoundary
        force[1] = J*(sin(X[2]-X[1])- sin(X[1]))
        force[N] = J*(sin(X[N-1]-X[N])-sin(X[N]))
    end
    for i in 1:N
        force[i] += -B*sin(X[i])
    end
    nothing
end
        
#=
Move the vector X a single time step dt, with the parameters J and sigma. "force" is used to store the force without allocating memory.
=#
function one_step(X::Vector{Float64},J::Float64,σ::Float64,dt::Float64, B::Float64, force::Vector{Float64}, boundary::BoundaryCondition)
    N=size(X)[1]
    noise = (rand(N) .- 0.5) .* (√12*σ*√dt)
    F(X,J,force,B,boundary)
    X .+= force .* dt .+ noise
    nothing
end
    
#=
Run the Ito process starting from X0 for time T with time discretization dt and parameters sigma,J
=#
function runprocess(X0::Vector{Float64},T::Float64,dt::Float64,σ::Float64,J::Float64, B::Float64, boundary::BoundaryCondition)
    N=size(X0)[1]
    X=X0
    steps = Int(ceil(T/dt))
    force = Array(X0)
    process = zeros(steps,N)
    for s in 1:steps
        one_step(X,J,σ,dt,B,force,boundary)
        process[s,:]=Array(X)
    end
    process
end


In [ ]:
N = 20
dt = 0.0001
T = 50.0
J = 40.0
σ = 3.0
B = 1.0

function simulate_magnetization(N,dt,T,σ,J,winding_number::Int,B,boundary::BoundaryCondition)
    X0 = [range(0,length = N, step = (winding_number+0.01)*2π/N)...]
    process = runprocess(X0,T,dt,σ,J,B,boundary)
    steps = size(process)[1]
    
    if boundary == PeriodicBoundary
        winding = [round(Int,windingnumber(process[i,:])) for i in 1:steps]
        if !all(winding .== winding_number)
            #println("System left the metastable state!")
            return N+1
        end
    end
    
    magnetization = [total_magnetization(process[i,:]) for i in 1:steps]
    return mean(magnetization[round(Int,steps/2):steps])
end

In [ ]:
magnetization = [simulate_magnetization(N,dt,T,σ,J,1,B,PeriodicBoundary) for i in range(1,100)]
magnetization = magnetization[real.(magnetization) .< N]
println(abs(mean(magnetization))," exp(",angle(mean(magnetization))," i)")
#histogram(real.(magnetization))

In [ ]:
magnetization = [simulate_magnetization(N,dt,T,σ,J,0,B,PeriodicBoundary) for i in range(1,100)]
magnetization = magnetization[real.(magnetization) .< N]
println(abs(mean(magnetization))," exp(",angle(mean(magnetization))," i)")
#histogram(real.(magnetization))

In [ ]:
magnetization = [simulate_magnetization(N,dt,T,σ,J,1,B,FreeBoundary) for i in range(1,100)]
magnetization = magnetization[real.(magnetization) .< N]
println(abs(mean(magnetization))," exp(",angle(mean(magnetization))," i)")
#histogram(real.(magnetization))

In [ ]:
magnetization = [simulate_magnetization(N,dt,T,σ,J,0,B,FreeBoundary) for i in range(1,100)]
magnetization = magnetization[real.(magnetization) .< N]
println(abs(mean(magnetization))," exp(",angle(mean(magnetization))," i)")
#histogram(real.(magnetization))